<div class="header">
  <img src="img/kg_logo_white_side.png" alt="logo" style="width: 300px;"/>
  <h1>03 Real-time Inferencing</h1>
</div>

Now that we have deployed the endpoint on GCP, we can perform the real-time inferencing. We follow these steps: 

1. Get transaction features - normally these would come from the client when the new transaction arrives. We will pull this from the feature store historical data
2. Query account embedding
3. Submit features to the model endpoint for inferencing

## Get transaction features

Normally this would come directly from the client (e.g. when a new transaction is received)

In [1]:
pipeline_uri = "0c57e63e49134502a0b2813c9a0e6d49"
mapping_table = "katana-clusters-beta.fsi_elliptic.account_mapping_"+pipeline_uri
features = "katana-clusters-beta.fsi_elliptic.account_features"

from google.cloud import bigquery
client = bigquery.Client()
sql = f"""
SELECT af.*
FROM 
    {mapping_table} am
JOIN {features} af
on am.account_id = af.account_id
WHERE am.account_gid = 82252
"""
df = client.query(sql).to_dataframe()


In [2]:
df.head()

,account_id,local_feat_2,local_feat_3,local_feat_4,local_feat_5,local_feat_6,local_feat_7,local_feat_8,local_feat_9,local_feat_10,...,local_feat_86,local_feat_87,local_feat_88,local_feat_89,local_feat_90,local_feat_91,local_feat_92,local_feat_93,local_feat_94,event_timestamp
0,248922756,-0.170438,-0.158783,-1.201369,-0.046932,-0.043875,-0.02914,-0.061584,-0.162775,-0.167751,...,1.347969,1.12559,1.128038,-0.287652,2.694074,3.482573,2.363482,1.135523,1.135279,2022-10-10 00:00:00+00:00


In [3]:
df = df.drop(columns= ['account_id','event_timestamp'])

In [4]:
local_features = df.to_dict()

In [5]:
local_features

{'local_feat_2': {0: -0.1704381951681886},
 'local_feat_3': {0: -0.1587825236260844},
 'local_feat_4': {0: -1.2013688016765636},
 'local_feat_5': {0: -0.0469320913859779},
 'local_feat_6': {0: -0.0438745479173489},
 'local_feat_7': {0: -0.0291397707181635},
 'local_feat_8': {0: -0.0615837940730322},
 'local_feat_9': {0: -0.1627752686438751},
 'local_feat_10': {0: -0.1677505998619076},
 'local_feat_11': {0: -0.0419301227905048},
 'local_feat_12': {0: -0.1646486225582168},
 'local_feat_13': {0: -2.516704748133169},
 'local_feat_14': {0: -2.486106024623408},
 'local_feat_15': {0: -0.0429552992580282},
 'local_feat_16': {0: -0.0132816148700588},
 'local_feat_17': {0: -0.0434141589424491},
 'local_feat_18': {0: -0.1695291606220958},
 'local_feat_19': {0: -0.1721881804459036},
 'local_feat_20': {0: -0.1732518414045185},
 'local_feat_21': {0: -1.373657177393896},
 'local_feat_22': {0: -1.37145982760274},
 'local_feat_23': {0: -0.1392989337184025},
 'local_feat_24': {0: 3.353529615906253},
 'l

In [6]:
embeddings_table = "katana-clusters-beta.fsi_elliptic.elliptic_embeddings_"+pipeline_uri
sql = f"""
SELECT af.*
FROM 
    {mapping_table} am
JOIN {embeddings_table} af
on am.account_gid = af.account_gid
WHERE am.account_gid = 82252
"""
embeddings = client.query(sql).to_dataframe()

In [7]:
embeddings.head()

,account_gid,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,embed_10,embed_11,embed_12,embed_13,embed_14,embed_15,event_timestamp
0,82252,0.0,0.0,0.29654,2.080075,0.1527,0.0,0.0,1.785956,0.0,0.499765,0.654764,0.0,0.055952,0.0,1.671464,0.0,2022-09-23 00:00:00+00:00


In [8]:
embeddings = embeddings.drop(columns= ['account_gid','event_timestamp'])

In [9]:
account_embeddings = embeddings.to_dict()

In [10]:
account_embeddings

{'embed_0': {0: 0.0},
 'embed_1': {0: 0.0},
 'embed_2': {0: 0.29653966426849365},
 'embed_3': {0: 2.0800745487213135},
 'embed_4': {0: 0.15269994735717773},
 'embed_5': {0: 0.0},
 'embed_6': {0: 0.0},
 'embed_7': {0: 1.785955548286438},
 'embed_8': {0: 0.0},
 'embed_9': {0: 0.49976465106010437},
 'embed_10': {0: 0.6547637581825256},
 'embed_11': {0: 0.0},
 'embed_12': {0: 0.05595153197646141},
 'embed_13': {0: 0.0},
 'embed_14': {0: 1.6714638471603394},
 'embed_15': {0: 0.0}}

In [11]:
account_embeddings.update(local_features)

In [12]:
account_embeddings

{'embed_0': {0: 0.0},
 'embed_1': {0: 0.0},
 'embed_2': {0: 0.29653966426849365},
 'embed_3': {0: 2.0800745487213135},
 'embed_4': {0: 0.15269994735717773},
 'embed_5': {0: 0.0},
 'embed_6': {0: 0.0},
 'embed_7': {0: 1.785955548286438},
 'embed_8': {0: 0.0},
 'embed_9': {0: 0.49976465106010437},
 'embed_10': {0: 0.6547637581825256},
 'embed_11': {0: 0.0},
 'embed_12': {0: 0.05595153197646141},
 'embed_13': {0: 0.0},
 'embed_14': {0: 1.6714638471603394},
 'embed_15': {0: 0.0},
 'local_feat_2': {0: -0.1704381951681886},
 'local_feat_3': {0: -0.1587825236260844},
 'local_feat_4': {0: -1.2013688016765636},
 'local_feat_5': {0: -0.0469320913859779},
 'local_feat_6': {0: -0.0438745479173489},
 'local_feat_7': {0: -0.0291397707181635},
 'local_feat_8': {0: -0.0615837940730322},
 'local_feat_9': {0: -0.1627752686438751},
 'local_feat_10': {0: -0.1677505998619076},
 'local_feat_11': {0: -0.0419301227905048},
 'local_feat_12': {0: -0.1646486225582168},
 'local_feat_13': {0: -2.516704748133169},


In [ ]:
df = store.get_historical_features(
    entity_df=f"""
    SELECT 
        account_gid,
        account_id,
        event_timestamp
    FROM 
         {mapping_table}   
    WHERE 
        account_gid = 82252
    """,
    features=store.get_feature_service("account_features_fs"),
    full_feature_names=False
).to_df()
df = df[df["account_gid"]==82252]
transaction_features = {f"local_feat_{i}": df[f"local_feat_{i}"].values[0] for i in range(2,95)}

## Real-time account embedding retrieval

Once we get the account ID that the transaction is for, we can look up the account embedding in real-time

In [4]:
account_embedding = store.get_online_features(
    features=[f"account_embeddings_fv:embed_{i}" for i in range(16)],
    entity_rows=[{"account_gid": "82252"}]
).to_dict()
account_embedding
account_embedding = {i: account_embedding[i][0] for i in account_embedding.keys()}
del account_embedding["account_gid"]
account_embedding.update(transaction_features)

## Real-time inferencing from model endpoint

In [15]:
from typing import Dict

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

project="936152536067"
endpoint_id="3775146785689829376"
location="us-central1"
api_endpoint= "us-central1-aiplatform.googleapis.com"
instance_dict=account_embedding
# The AI Platform services require regional API endpoints.
client_options = {"api_endpoint": api_endpoint}
# Initialize client that will be used to create and send requests.
# This client only needs to be created once, and can be reused for multiple requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
# for more info on the instance schema, please use get_model_sample.py
# and look at the yaml found in instance_schema_uri
instance = json_format.ParseDict(instance_dict, Value())
instances = [instance]
parameters_dict = {}
parameters = json_format.ParseDict(parameters_dict, Value())
endpoint = client.endpoint_path(
    project=project, location=location, endpoint=endpoint_id
)
response = client.predict(
    endpoint=endpoint, instances=instances, parameters=parameters
)
print("response")
print(" deployed_model_id:", response.deployed_model_id)
# See gs://google-cloud-aiplatform/schema/predict/prediction/tabular_classification_1.0.0.yaml for the format of the predictions.
predictions = response.predictions
for prediction in predictions:
    print(" prediction:", dict(prediction))

ContextualVersionConflict: (google-cloud-bigquery 3.2.0 (/opt/conda/lib/python3.8/site-packages), Requirement.parse('google-cloud-bigquery<3.0.0dev,>=1.15.0'), {'google-cloud-aiplatform'})

In [14]:
pip install google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.4/408.4 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0rc2
    Uninstalling protobuf-3.20.0rc2:
      Successfully uninstalled protobuf-3.20.0rc2
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.2.0
    Uninstalling google-cloud-bigquery-3.2.0:
      Successfully uninstalled google-cloud-bigquery-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboardx 2.5.1 requires pr